In [17]:
# pip install playwright

In [39]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

In [40]:
async def scrape_jobstreet():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://ph.jobstreet.com/Data-Science-jobs?sortmode=ListedDate")

        print("🔍 Scraping listing page...")
        await asyncio.sleep(5)  # let page fully load

        jobs = await page.query_selector_all('article[data-testid="job-card"]')
        print(f"📄 Found {len(jobs)} job listings")

        data = []

        for job in jobs:
            try:
                title = await job.get_attribute("aria-label")
                title = title.strip() if title else "N/A"

                anchor = await job.query_selector("a[data-automation='job-list-item-link-overlay']")
                url_suffix = await anchor.get_attribute("href") if anchor else "N/A"
                job_url = "https://ph.jobstreet.com" + url_suffix if url_suffix and url_suffix.startswith("/job") else url_suffix

                # Extract quick visible details
                all_spans = await job.query_selector_all("span")
                text_items = []
                for span in all_spans:
                    try:
                        txt = await span.inner_text()
                        if txt.strip():
                            text_items.append(txt.strip())
                    except:
                        continue

                company = text_items[0] if len(text_items) > 0 else "N/A"
                location = next((t for t in text_items if "Metro Manila" in t or "City" in t), "N/A")
                salary = next((t for t in text_items if "₱" in t or "PHP" in t), "N/A")
                posted = next((t for t in text_items if "ago" in t or "day" in t or "hour" in t), "N/A")

                # Visit job detail page for full description
                job_description = "N/A"
                if job_url != "N/A":
                    detail_page = await browser.new_page()
                    await detail_page.goto(job_url)
                    await asyncio.sleep(2)

                    try:
                        desc_el = await detail_page.query_selector("div[data-automation='jobAdDetails']")
                        job_description = await desc_el.inner_text() if desc_el else "N/A"
                    except:
                        pass
                    await detail_page.close()

                data.append({
                    "Job Title": title,
                    "Company": company,
                    "Location": location,
                    "Expected Salary": salary,
                    "Posted": posted,
                    "Job URL": job_url,
                    "Description": job_description
                })

            except Exception as e:
                print("⚠️ Error parsing job:", e)

        await browser.close()
        return pd.DataFrame(data)


In [41]:
# Run the async function inside Jupyter
df = await scrape_jobstreet()
df.head()

🔍 Scraping listing page...
📄 Found 32 job listings


,Job Title,Company,Location,Expected Salary,Posted,Job URL,Description
0,SOC L3 Analyst,at\nCEVA IT SHARED SERVICE,"Makati City, Metro Manila(Hybrid)",N/A,6m ago,https://ph.jobstreet.com/job/85843406?type=sta...,YOUR ROLE\n\nWe are looking for a Security Ope...
1,Reporting Analyst | Makati | URGENT!,"at\nPeak Outsourcing, Inc.","Makati City, Metro Manila",N/A,13m ago,https://ph.jobstreet.com/job/85843368?type=sta...,Reporting Analyst job description\n\nWe are lo...
2,CRM & Workflow Automation Specialist (Permanen...,at\nIntrinsic Digital,"Manila City, Metro Manila(Remote)","₱65,000 – ₱75,000 per month",32m ago,https://ph.jobstreet.com/job/85843302?type=sta...,We are a US-based mobile marketing agency hiri...
3,Database Administrator (SQL Server & Azure Dat...,at\niSupport Worldwide,"Ortigas, Metro Manila(Remote)",N/A,46m ago,https://ph.jobstreet.com/job/85843225?type=sta...,Be a part of our fast-growing team and unchain...
4,Quality Test Engineer (SQL Scripting),at\nREED ELSEVIER SHARED SERVICES (PHILIPPINES...,Metro Manila(Hybrid),N/A,1h ago,https://ph.jobstreet.com/job/85842965?type=sta...,Responsibility:\n\nTo understand product requi...


In [42]:
display(df)

,Job Title,Company,Location,Expected Salary,Posted,Job URL,Description
0,SOC L3 Analyst,at\nCEVA IT SHARED SERVICE,"Makati City, Metro Manila(Hybrid)",N/A,6m ago,https://ph.jobstreet.com/job/85843406?type=sta...,YOUR ROLE\n\nWe are looking for a Security Ope...
1,Reporting Analyst | Makati | URGENT!,"at\nPeak Outsourcing, Inc.","Makati City, Metro Manila",N/A,13m ago,https://ph.jobstreet.com/job/85843368?type=sta...,Reporting Analyst job description\n\nWe are lo...
2,CRM & Workflow Automation Specialist (Permanen...,at\nIntrinsic Digital,"Manila City, Metro Manila(Remote)","₱65,000 – ₱75,000 per month",32m ago,https://ph.jobstreet.com/job/85843302?type=sta...,We are a US-based mobile marketing agency hiri...
3,Database Administrator (SQL Server & Azure Dat...,at\niSupport Worldwide,"Ortigas, Metro Manila(Remote)",N/A,46m ago,https://ph.jobstreet.com/job/85843225?type=sta...,Be a part of our fast-growing team and unchain...
4,Quality Test Engineer (SQL Scripting),at\nREED ELSEVIER SHARED SERVICES (PHILIPPINES...,Metro Manila(Hybrid),N/A,1h ago,https://ph.jobstreet.com/job/85842965?type=sta...,Responsibility:\n\nTo understand product requi...
5,ServiceNow Developer,at\nREED ELSEVIER SHARED SERVICES (PHILIPPINES...,"Quezon City, Metro Manila(Hybrid)",N/A,1h ago,https://ph.jobstreet.com/job/85842842?type=sta...,Job Summary:\n\nThe ideal candidate will have ...
6,Alabang | Business Intelligence Analyst,at\nTeleperformance,"Alabang, Metro Manila",N/A,2h ago,https://ph.jobstreet.com/job/85842276?type=sta...,Teleperformance is looking for a detail-orient...
7,Reports Analyst | Power BI Developer | Onsite ...,at\nFoundever®,"Quezon City, Metro Manila",N/A,3h ago,https://ph.jobstreet.com/job/85842231?type=sta...,Position Overview\n\nIncumbents may perform so...
8,SAP Data Archiving Consultant,at\nPrivate Advertiser,"Manila City, Metro Manila(Hybrid)","₱100,000 – ₱120,000 per month",3h ago,https://ph.jobstreet.com/job/85842225?type=sta...,Job Summary:\n\nWe are looking for an experien...
9,Operations Associate- Certificates Renewals,at\nATOS Information Technology Inc.,"Taguig City, Metro Manila(Hybrid)",N/A,HMO effective day 1,https://ph.jobstreet.com/job/85842182?type=sta...,About Atos\n\nAtos is a global leader in digit...
